## Import Dependencies 

In [1]:
import sys

# sys.path.insert(0, '../MIR')

In [2]:
from src.utils.construct_positional_indexes import construct_positional_indexes
from src.utils.load_index import load_index
from src.models.TextPreparer import TextPreparer
from src.enums import *

In [4]:
manager = construct_positional_indexes()

100%|██████████| 1572/1572 [00:21<00:00, 74.19it/s] 


In [5]:
manager.save_index('manager.pickle')

In [3]:
manager = load_index('manager.pickle')

In [4]:
manager.corpus_index.get_posting_list('با')[0].get_positions(Fields.TEXT)

[424, 465, 485, 659]

In [5]:
query = "نظرخواهی انجام شده توسط دانشگاه دانشگاه"
# method = Methods.LTC_LNC
weights = [1., 2.]

In [5]:
manager.add_document_to_indexes('./data/Add.xml', 7157)

In [6]:
manager.search({Fields.TITLE:'"آقا علیرضا"', Fields.TEXT: 'علیرضا'})

[7157]

In [17]:
manager.documents[3197].title

'تخت جمشید'

In [15]:
manager.search(query)

[6791,
 3897,
 5428,
 3921,
 4313,
 3919,
 5766,
 4275,
 3666,
 4058,
 5237,
 5293,
 5967,
 5509,
 3874]

In [7]:
manager.corpus_index.index['.'].doc_frequency

{'title': 9, 'text': 1280}

In [11]:
manager.delete_document_from_indexes('./data/Add.xml', 7157)

In [12]:
print(manager.corpus_index.index['آقا'].posting_list[-1])


        Document ID: 7153
        Title Positions: []
        Text Positions: [1513, 1723, 1835]
        


In [16]:
manager.corpus_index.index['آقا'].doc_frequency

{'title': 1, 'text': 61}

In [1]:
import numpy as np

from src.utils.read_queries import read_queries
from src.metrics import f_measure, precision, r_precision, recall, ndcg_at_k, average_precision

In [2]:
average_precision([1,0,1,0,1])

0.7555555555555555

In [6]:
MAX_RETRIEVED = 15

In [3]:
def evaluate_search_engine(query_id="all"):
    queries, relevants = read_queries(query_id)
    results_r_precision = []
    results_ndcg = []
    results_f_measure = []
    results_map = []
    for query, query_relevants in zip(queries, relevants):
        num_relevant_docs = len(query_relevants)
        retrieved_docs = manager.search(query, k=num_relevant_docs)
        relevance = []
        for retrieved_doc in retrieved_docs:
            if retrieved_doc in query_relevants:
                relevance.append(1)
            else:
                relevance.append(0)
        results_r_precision.append(r_precision(relevance, num_relevant_docs))
        results_ndcg.append(ndcg_at_k(relevance, num_relevant_docs))
        results_f_measure.append(f_measure(relevance, num_relevant_docs))
        results_map.append(average_precision(relevance))
    return np.mean(results_r_precision), np.mean(results_ndcg), np.mean(results_f_measure), np.mean(results_map)

In [7]:
evaluate_search_engine()

(0.6723282130404556,
 0.7058278702806886,
 0.6723282130404556,
 0.8153515319937519)